In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import time

Using TensorFlow backend.


# 取得training data 

In [2]:
# Get images(training的input(IR照片))
X = []
for filename in os.listdir('container1012/IR_resize/'):
    X.append(img_to_array(load_img('container1012/IR_resize/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

# Get images(training的label(白天照片))
Y = []
for filename in os.listdir('container1012/daylight_resize/'):
    Y.append(img_to_array(load_img('container1012/daylight_resize/'+filename)))
Y = np.array(Y, dtype=float)
Ytrain = 1.0/255*Y

#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

# 如果出現 類似'data / training / images / .ipynb_checkpoints'的error 

In [17]:
!rm -rf test1012/IRstream_resize_gray/.ipynb_checkpoints

In [3]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

# 設定參數

In [4]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
#         shear_range=0.2,
#         zoom_range=0.2,
#         rotation_range=20,
#         horizontal_flip=True
)

#Generate training data
batch_size = 1

def image_a_b_gen(batch_size): 
    for batch,batch_label in zip(datagen.flow(Xtrain, batch_size=batch_size),datagen.flow(Ytrain, batch_size=batch_size)):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        ###########################################################以下是讀label的兩個通道
        lab_batch_label = rgb2lab(batch_label)
        Y_batch = lab_batch_label[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

# 訓練

In [7]:
#Train model      
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), epochs=500, steps_per_epoch=1)

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 6.2027e-05
Epoch 2/500
1/1 [==============================] - 0s 156ms/step - loss: 0.0015
Epoch 3/500
1/1 [==============================] - 0s 187ms/step - loss: 6.9488e-04
Epoch 4/500
1/1 [==============================] - 0s 203ms/step - loss: 7.9724e-04
Epoch 5/500
1/1 [==============================] - 0s 173ms/step - loss: 7.3389e-04
Epoch 6/500
1/1 [==============================] - 0s 160ms/step - loss: 5.8981e-04
Epoch 7/500
1/1 [==============================] - 0s 171ms/step - loss: 4.8030e-04
Epoch 8/500
1/1 [==============================] - 0s 182ms/step - loss: 4.4083e-04
Epoch 9/500
1/1 [==============================] - 0s 207ms/step - loss: 4.1050e-04
Epoch 10/500
1/1 [==============================] - 0s 213ms/step - loss: 4.0511e-04
Epoch 11/500
1/1 [==============================] - 0s 196ms/step - loss: 3.7354e-04
Epoch 12/500
1/1 [==============================] - 0s 188ms/step - loss: 3.3806e

In [8]:
model.save('container1012/1012_model.h5')  # creates a HDF5 file 'my_model.h5'

In [9]:
from keras.models import load_model
model = load_model('container1012/1012_model.h5')

# 執行預測(上色)

In [21]:
start = time.time()
color_me = []
#以下輸入灰階照片(待上色)
for filename in os.listdir('test1012/IRstream_resize_gray/'):
    color_me.append(img_to_array(load_img('test1012/IRstream_resize_gray/'+filename)))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("test1012/IRcolor_img_resize/img_"+str(i)+".jpg", lab2rgb(cur))
end = time.time() 
print("執行時間：%f 秒" % (end - start))
    

執行時間：15.558220 秒


In [ ]:
!rm -rf test1012/IRstream_resize_gray/.ipynb_checkpoints